In [2]:
import numpy as np 
import pandas as pd 

countries_report = pd.read_csv("countries.csv", header = 0, low_memory=False)
happiness_report = pd.read_csv("2017.csv", header = 0, low_memory=False)

happiness_report = happiness_report.sample(frac=1)
print happiness_report.shape
print countries_report.shape

(151, 12)
(227, 20)


In [3]:
happy_data = []
country_data = []

rows = happiness_report.shape[0]
for i in range(rows):
    key = unicode("Country", "utf-8")
    c = happiness_report.loc[happiness_report.index[[i]], key]
    c_data = happiness_report.loc[happiness_report.index[[i]]]
#     print c_data.values[0]
    happy_data.append(c_data.values[0])
    
    # Gets corresponding countries.csv data for each country in 2017.ccsv
    Country = happiness_report.loc[happiness_report.index[[i]], key].values
#     print Country[0]
    
    col = countries_report['Country']
    index = -1
    for i in col:
        index += 1
        i = str(i)
        if Country[0] in i:
#             print i
            idx = index
            break
        
    data = countries_report.loc[countries_report.index[idx]]
    country_data.append(data)

In [4]:
print len(happy_data)
print len(country_data)

#sanity check
idx = 0
for i in country_data:
    print happy_data[idx][0],
    print i[0]
    idx += 1

151
151
United States United States 
Austria Austria 
Haiti Haiti 
Croatia Croatia 
India India 
Senegal Senegal 
Kyrgyzstan Kyrgyzstan 
Bangladesh Bangladesh 
Mauritius Mauritius 
Namibia Namibia 
Saudi Arabia Saudi Arabia 
Somalia Somalia 
Latvia Latvia 
Liberia Liberia 
Nicaragua Nicaragua 
Netherlands Netherlands 
Colombia Colombia 
Mexico Mexico 
Burkina Faso Burkina Faso 
Cote d'Ivoire  Cote d'Ivoire 
Guatemala Guatemala 
Armenia Armenia 
Belgium Belgium 
Sweden Sweden 
Argentina Argentina 
Uruguay Uruguay 
Hong Kong Hong Kong 
South Korea South Korea 
Tunisia Tunisia 
Lebanon Lebanon 
Rwanda Rwanda 
Egypt Egypt 
Trinidad and Tobago Trinidad and Tobago 
Moldova Moldova 
El Salvador El Salvador 
Iceland Iceland 
Botswana Botswana 
Uganda Uganda 
Ukraine Ukraine 
Nepal Nepal 
Zambia Zambia 
Sri Lanka Sri Lanka 
Benin Benin 
Costa Rica Costa Rica 
Mongolia Mongolia 
Honduras Honduras 
Canada Canada 
Lithuania Lithuania 
Norway Norway 
France France 
Mauritania Mauritania 
Serbia Ser

In [5]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# print happy_data
happyX = []
happyY = []

for i in happy_data:
    happyX.append(i[0])
    happyY.append(i[2])
    
happyX = np.array(happyX)
happyY = np.array(happyY)
print happyX.shape
print happyY.shape

def mse_baseline(y_train, y_test):
    avg = np.average(y_train)
    print avg
    y_predict = np.full(y_test.shape, avg)
    print "baseline", mean_squared_error(y_test, y_predict)

kf = KFold(n_splits=3)
for train_index, test_index in kf.split(happyX):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = happyX[train_index], happyX[test_index]
    y_train, y_test = happyY[train_index], happyY[test_index]
    mse_baseline(y_train, y_test)

(151,)
(151,)
5.32090000866
baseline 1.43071929273
5.40611881788
baseline 1.2078043056
5.3743762238
baseline 1.2191023676


In [6]:
import math
allArr = np.zeros(19)
for i in country_data:
    for j in range(2,19):
        if isinstance(i[j], float) or isinstance(i[j], int):
            if not math.isnan(i[j]):
                allArr[j] += i[j]
        else:
            allArr[j] +=(float(i[j].replace(',','.')))
means = allArr/float(len(country_data))

In [8]:
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

countriesX = [] 

regions = []
for i in country_data:
    regions.append(i[1])

le = preprocessing.LabelEncoder()
regions_codes = le.fit_transform(regions)

for num, i in enumerate(country_data):
    info = []
    info.append(regions_codes[num])
    for j in range(2,19): 
        if isinstance(i[j], float) or isinstance(i[j], int):
            if not math.isnan(i[j]):
                info.append(i[j])
            else:
                info.append(means[j])
        else:
            info.append(float(i[j].replace(',','.')))  
    countriesX.append(info)
    
countriesX = np.array(countriesX)


def mse_clf(X_train, X_test, y_train, y_test):
    feat = SelectKBest(k=3, score_func=mutual_info_regression)
    feat.fit(X_train, y_train)
    scores = feat.scores_
    print sorted(range(len(scores)), key=lambda i: scores[i])[-3:]
    X_train_new = feat.transform(X_train)
    X_test_new = feat.transform(X_test)

kf = KFold(n_splits=5)
for train_index, test_index in kf.split(countriesX):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = countriesX[train_index], countriesX[test_index]
    y_train, y_test = happyY[train_index], happyY[test_index]
    
    kfInternal = KFold(n_splits=5)
    for train_2, test_2 in kf.split(X_train):
        X_2_train, X_2_test = X_train[train_2], X_train[test_2]
        y_2_train, y_2_test = y_train[train_2], y_train[test_2]

    mse_clf(X_2_train, X_2_test, y_2_train, y_2_test)

[9, 6, 7]
[16, 7, 6]
[9, 6, 7]
[9, 7, 6]
[0, 7, 6]
